In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
dataset1 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
dataset2 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
dataset3 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
dataset4 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
dataset5 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
dataset6 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
dataset7 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
dataset8 = pd.read_csv("../MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

In [3]:
df_list = [dataset1,dataset2,dataset3,dataset4,dataset5,dataset6,dataset7,dataset8]
del dataset1
del dataset2
del dataset3
del dataset4
del dataset5
del dataset6
del dataset7
del dataset8
dataset = pd.concat(df_list, ignore_index=True)
del df_list

In [4]:
dataset.at[65, ' Label']

'BENIGN'

In [5]:
dataset.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
# removing rows with empty values and turn all attacks into one label
dataset.replace([np.inf,-np.inf], np.nan, inplace=True)
dataset.replace(['DoS Hulk',
                 'PortScan',
                 'DDoS',
                 'DoS GoldenEye',
                 'FTP-Patator',
                 'SSH-Patator',
                 'DoS slowloris',
                 'DoS Slowhttptest',
                 'Bot',
                 'Web Attack � Brute Force',
                 'Web Attack � XSS',
                 'Infiltration',
                 'Web Attack � Sql Injection',
                 'Heartbleed'], 'attack', inplace=True)
df = dataset.dropna(axis=0).reset_index(drop=True)
del dataset

In [7]:
df[' Label'].value_counts()

BENIGN    2271320
attack     556556
Name:  Label, dtype: int64

In [9]:

df.replace('BENIGN', 0, inplace=True)
df.replace('attack', 1, inplace=True)



In [9]:
import tensorflow as tf
from tensorflow.keras import layers


model = tf.keras.Sequential()

model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:].values, df.iloc[:, -1].values, test_size=0.2, random_state=0)

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("latesatmodel.h5",
                                 monitor="val_loss",
                                 mode="min",
                                 save_best_only=True,
                                 verbose=1)

  # EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor='val_loss',
                            restore_best_weights=True,
                            patience=10,  # number of epochs with
                            # no improvement after which
                            # training will be stopped
                            verbose=1)

In [15]:
history = model.fit(x_train,y_train, epochs=10, callbacks=[checkpoint,earlystop],  batch_size=32)

Epoch 1/10
70697/70697 [==============================] - 415s 6ms/step - loss: 7.2498e-04 - accuracy: 0.9998
Epoch 2/10
70697/70697 [==============================] - 455s 6ms/step - loss: 6.8201e-05 - accuracy: 1.0000
Epoch 3/10
70697/70697 [==============================] - 410s 6ms/step - loss: 4.5994e-06 - accuracy: 1.0000
Epoch 4/10
70697/70697 [==============================] - 409s 6ms/step - loss: 5.4561e-06 - accuracy: 1.0000
Epoch 5/10
70697/70697 [==============================] - 417s 6ms/step - loss: 6.5524e-06 - accuracy: 1.0000
Epoch 6/10
70697/70697 [==============================] - 409s 6ms/step - loss: 9.4742e-06 - accuracy: 1.0000
Epoch 7/10
70697/70697 [==============================] - 401s 6ms/step - loss: 8.0406e-06 - accuracy: 1.0000
Epoch 8/10
70697/70697 [==============================] - 411s 6ms/step - loss: 8.8512e-06 - accuracy: 1.0000
Epoch 9/10
70697/70697 [==============================] - 423s 6ms/step - loss: 1.5814e-05 - accuracy: 1.0000
Epoch 10/1

In [16]:
model.save("basic_seq_binary.h5")

In [17]:
model.evaluate(x_test,y_test)

17675/17675 [==============================] - 61s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [10]:
#combined metric
from keras import backend as K

acc_w = 0.25
f1_w = 0.25
pre_w = 0.25
rec_w = 0.25


def combined_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_score = 2*(precision*recall)/(precision+recall+K.epsilon())
    accuracy = K.mean(K.equal(y_true, K.round(y_pred)))
    return (accuracy*acc_w+f1_score*f1_w+precision*pre_w+recall*rec_w)/4


In [11]:
df.at[65, ' Label']


0

In [12]:
import gym
from gym import spaces
import random

class MyEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, data):
        self.action_space = spaces.Discrete(2) # 2 possible actions
        self.observation_space = spaces.Discrete(1) # only one observation needed
        self.reward_range = (-1, 1) # rewards range from -1 to 1
        self.data = data # dataset
        self.index = 0 # index of the current data point
        self.state = None
        self.done = False

    def step(self, action):
        label = self.data.at[self.index, ' Label'] # get label from dataset
        if action == label:
            reward = 1 # match, give positive reward
        else:
            reward = -1 # mismatch, give negative reward
        self.index += 1 # move to next data point
        if self.index == len(self.data):
            self.done = True # end of dataset
        self.state = self.data.iloc[self.index,:-1].values.reshape(1,78)
        return self.state, reward, self.done, {}

    def reset(self):
        self.index = 0 # reset index
        self.done = False
        self.state = self.data.iloc[self.index,:-1].values.reshape(1,78)
        return self.state

    def render(self, mode='human', close=False):
        pass


In [1]:
env = MyEnv(df.sample(frac=0.1))




NameError: name 'MyEnv' is not defined

In [14]:
state = env.reset()
print(state.shape)

(1, 78)


In [15]:
action = env.action_space.sample()
print(action)

1


In [16]:
next_state, reward, done, info = env.step(action)
print(reward)
print(done)

-1
False


In [22]:
from collections import deque
from tensorflow import keras
from tensorflow.keras import layers

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = keras.Sequential()
        model.add(layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(layers.Dense(24, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=self.learning_rate), metrics=["Accuracy"])
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [24]:
from sklearn import preprocessing

EPISODES= 10
state_size= 78
action_size=2

agent = DQNAgent(state_size, action_size)
batch_size = 32


for e in range(EPISODES):
    state = env.reset()
    #print(state.shape)
    #state = state.values.reshape(1, state_size)
    print(state.shape)
    for time in range(300000):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        #next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                    .format(e, EPISODES, time, agent.epsilon))
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


(1, 78)


KeyboardInterrupt: 

In [21]:
agent.save("dqn_model.h5")